In [6]:
from transformers import AutoConfig, AutoTokenizer,TFBertTokenizer, BertTokenizer,BertTokenizerFast,BertForTokenClassification #pytorch packageBertTokenizerFast
import torch

In [7]:
#folder path
model_path = "/Users/u6022257/Documents/Transformers/models/biobert_genetic_ner/"
tokenizer_config = model_path+"tokenizer_config.json"
model_config = model_path+"config.json"
vocab_file = model_path+"vocab.txt"

In [9]:
config_model = AutoConfig.from_pretrained(model_config)
tokenizer = BertTokenizerFast(vocab_file,do_lower_case = True)
#tokenizer = AutoTokenizer.from_pretrained(model_path,use_fast=True)#AutoTokenizer does not work for bert genetic detection
#tokenizer = BertTokenizer(vocab_file,do_lower_case = True)
Model = BertForTokenClassification.from_pretrained(model_path, local_files_only=True)
tokenizer.is_fast

True

In [10]:
id2label = {
    "0": "B-GENETIC",
    "1": "I-GENETIC",
    "2": "O"
  }
label2id = {
    "B-GENETIC": 0,
    "I-GENETIC": 1,
    "O": 2
  }

In [11]:
sampleText = "This patient is heterozygous in the PPP2R5D gene for a de novo variant designated c.598G>A, which is predicted to result in the amino acid substitution p.Glu200Lys. This is a recurrent de novo variant that has been reported to be causative for autosomal dominant intellectual disability (Table 1, Houge et al. 2015. PubMed ID: 26168268; Table 1, Loveday et al. 2015. PubMed ID: 25972378; Supp. Tables 6 and 10, Lelieveld et al. 2017. PubMed ID: 28867141; Table 1, Reijnders et al. 2017. PubMed ID: 29051493). In summary, we interpret this variant as pathogenic. Pathogenic variants in PPP2R5D are associated with autosomal dominant intellectual disability 35 (OMIM: #616355). Clinical features include increased height (variable), macrocephaly (variable), absent speech, dysmorphic facial features, delayed psychomotor development, and seizures. Variants are reported to occur de novo. Clinical correlation is recommended."

In [16]:
encode_dic = tokenizer.encode_plus(
                    sampleText,                      # input text to tokenize
                    return_tensors="pt",
                    truncation=True,
                    max_length=512,            # maximum sequence length
                    padding=True,    # pad shorter sequences
                    return_offsets_mapping=True,  # return offsets
)
inputs = tokenizer.encode(
    sampleText,
    return_tensors="pt",
    max_length=512,
    truncation=True,
    padding=True    # pad shorter sequences
)

tokens = tokenizer.tokenize(
    tokenizer.decode(
        tokenizer.encode(
            sampleText,
            #return_tensors="pt",
            max_length=512,
            truncation=True,
            padding=True
        )
    ))

offsets = encode_dic['offset_mapping']
offsetsList = offsets.tolist()
print(len(offsetsList[0]),len(tokens))
print(offsetsList)
print(tokens)
#for i in range(len(tokens)):
#    print(tokens[i],offsetsList[0][i])

265 271
[[[0, 0], [0, 4], [5, 12], [13, 15], [16, 18], [18, 22], [22, 24], [24, 26], [26, 28], [29, 31], [32, 35], [36, 38], [38, 39], [39, 40], [40, 41], [41, 42], [42, 43], [44, 48], [49, 52], [53, 54], [55, 57], [58, 60], [60, 62], [63, 70], [71, 81], [82, 83], [83, 84], [84, 86], [86, 87], [87, 88], [88, 89], [89, 90], [90, 91], [92, 97], [98, 100], [101, 110], [111, 113], [114, 120], [121, 123], [124, 127], [128, 133], [134, 138], [139, 151], [152, 153], [153, 154], [154, 155], [155, 157], [157, 159], [159, 160], [160, 162], [162, 163], [163, 164], [165, 169], [170, 172], [173, 174], [175, 177], [177, 184], [185, 187], [188, 190], [190, 192], [193, 200], [201, 205], [206, 209], [210, 214], [215, 223], [224, 226], [227, 229], [230, 232], [232, 235], [235, 239], [240, 243], [244, 248], [248, 250], [250, 253], [254, 262], [263, 275], [276, 286], [287, 288], [288, 293], [294, 295], [295, 296], [297, 299], [299, 302], [303, 305], [306, 308], [308, 309], [310, 314], [314, 315], [316, 31

In [19]:
outputs = Model(inputs)[0]
predictions = torch.argmax(outputs,dim=2)
print(predictions[0].tolist())
print(len(tokens),len(predictions[0].tolist()),len(offsetsList[0]))
labels = predictions[0].tolist()
#tOut = []
lOut = []
offsetsOut = [] 
for i in range(len(predictions[0].tolist())):
    lOut.append(id2label[str(labels[i])])
    offsetsOut.append(offsetsList[0][i])
    print(id2label[str(labels[i])],offsetsList[0][i])

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
271 265 265
O [0, 0]
O [0, 4]
O [5, 12]
O [13, 15]
O [16, 18]
O [18, 22]
O [22, 24]
O [24, 26]
O [26, 28]
O [29, 31]
O [32, 35]
B-GENETIC [36, 38]
I-GENETIC [38, 39]
I-GENETIC [39, 40]
I-GENETIC [40, 41]


In [24]:
offsetsOut[1][0]#starting span
offsetsOut[1][1]#Ending span
lOut

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-GENETIC',
 'I-GENETIC',
 'I-GENETIC',
 'I-GENETIC',
 'I-GENETIC',
 'I-GENETIC',
 'I-GENETIC',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 '

In [28]:
SpanList = []
for i in range(len(lOut)):
    if lOut[i]=='B-GENETIC':
        currentStart = offsetsOut[i][0]
        #initialize the endspan
        currentEnd = offsetsOut[i][1]
        j=1
        while(lOut[i+j]=='I-GENETIC'):
            currentEnd = offsetsOut[i+j][1]
            j=j+1
            
        print("current span:", currentStart, currentEnd)
        SpanList.append([currentStart,currentEnd])
        

current span: 36 48
current span: 585 592


In [33]:
sampleText[SpanList[0][0]:SpanList[0][1]]


'PPP2R5D gene'

In [34]:
sampleText[SpanList[1][0]:SpanList[1][1]]

'PPP2R5D'